In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_tcpstream = pd.read_csv('data/bench_tcpstream.csv', delimiter=';', )
df_file = pd.read_csv('data/bench_file.csv', delimiter=';')

In [ ]:
# Calculate the mean of each column for both dataframes
tcpstream_means = df_tcpstream.mean()
file_means = df_file.mean()


In [ ]:
tcpstream_means['io_time'] = tcpstream_means['read_io_time'] + tcpstream_means['write_io_time']
file_means['io_time'] = file_means['read_io_time'] + file_means['write_io_time']


In [ ]:
tcpstream_means['p2m_overhead'] =  tcpstream_means['read_io_request_req_time'] + tcpstream_means['write_io_request_req_time'] + tcpstream_means['read_io_request_ack_time'] + tcpstream_means['write_io_request_ack_time'] + tcpstream_means['read_io_report_req_time'] + tcpstream_means['write_io_report_req_time'] + tcpstream_means['read_io_report_ack_time'] + tcpstream_means['write_io_report_ack_time']
file_means['p2m_overhead'] = file_means['read_io_request_req_time'] + file_means['write_io_request_req_time'] + file_means['read_io_request_ack_time'] + file_means['write_io_request_ack_time'] + file_means['read_io_report_req_time'] + file_means['write_io_report_req_time'] + file_means['read_io_report_ack_time'] + file_means['write_io_report_ack_time']

In [ ]:
tcpstream_means['middleware_overhead'] = tcpstream_means['read_io_request_processing_time'] + tcpstream_means['write_io_request_processing_time'] + tcpstream_means['read_io_report_processing_time'] + tcpstream_means['write_io_report_processing_time']
file_means['middleware_overhead'] = file_means['read_io_request_processing_time'] + file_means['write_io_request_processing_time'] + file_means['read_io_report_processing_time'] + file_means['write_io_report_processing_time']

In [ ]:
tcpstream_means['m2m_overhead'] = tcpstream_means['write_reserve_req_time'] + tcpstream_means['write_reserve_ack_time'] + tcpstream_means['write_sync_prov_req_time'] + tcpstream_means['write_sync_prov_ack_time']

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.35  # width of the bars

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Std IO exec time
std_io_time_bars = np.array([file_means['io_time'], tcpstream_means['io_time']])
ax.bar(x - width/2, std_io_time_bars, width, label='Standard I/O Execution Time', color='lightsalmon')

# Stde2e IO exec time
stde2e_io_time_bars = np.array([
                        file_means['io_time'] + file_means['middleware_overhead'] + file_means['p2m_overhead'],
                        tcpstream_means['io_time'] + tcpstream_means['middleware_overhead'] + tcpstream_means['p2m_overhead']
                    ])
ax.bar(x + width/2, stde2e_io_time_bars, width, label='Trace2e I/O Execution Time (stde2e)', color='lightblue')

# Add labels, title and legend
ax.set_xlabel('Resource Type')
ax.set_ylabel('Time (microseconds)')
ax.set_title('Comparison of I/O Libraries Performance: Standard vs TracE2E')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add values on top of the bars
for i, v in enumerate(std_io_time_bars):
    ax.text(i - width/2, v, f'{v:.0f}', ha='center', va='bottom')
for i, v in enumerate(stde2e_io_time_bars):
    ax.text(i + width/2, v, f'{v:.0f}', ha='center', va='bottom')

# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.35  # width of the bars

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Stde2e IO exec time
stde2e_io_time_bars = np.array([
                        file_means['io_time'] + file_means['middleware_overhead'] + file_means['p2m_overhead'],
                        tcpstream_means['io_time'] + tcpstream_means['middleware_overhead'] + tcpstream_means['p2m_overhead']
                    ])
ax.bar(x - width*0.75, stde2e_io_time_bars, width*0.5, label='Total Trace2e I/O Execution Time', color='lightblue')
# For the stacked bars
# First layer: io_time
std_io_time_bars = np.array([file_means['io_time'], tcpstream_means['io_time']])
ax.bar(x + width*0.25, 
        std_io_time_bars, 
        width*1.5, 
        label='including Call to Standard Library', 
        color='lightsalmon')

# Second layer: middleware_overhead
middleware_overhead_bars = np.array([file_means['middleware_overhead'], tcpstream_means['middleware_overhead']])
ax.bar(x + width*0.25, 
        middleware_overhead_bars, 
        width*1.5, 
        bottom=[file_means['io_time'], tcpstream_means['io_time']], 
        label='including Middleware Overhead', 
        color='mediumaquamarine')

# Third layer: p2m_overhead
p2m_overhead_bars = np.array([file_means['p2m_overhead'], tcpstream_means['p2m_overhead']])
ax.bar(x + width*0.25, 
    p2m_overhead_bars, 
    width*1.5, 
    bottom=std_io_time_bars+middleware_overhead_bars, 
    label='including P2M Overhead', 
    color='gold')

# Add labels, title and legend
middle_y = std_io_time_bars[i] + middleware_overhead_bars[i] + v*0.5
ax.text(i + width*0.25, middle_y, f'{v:.0f}', ha='center', va='center', color='black')

ax.set_xlabel('Resource Type')
ax.set_ylabel('Time (microseconds)')
ax.set_title('TracE2E I/O Overhead Analysis')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add values in the middle of the bars
for i, v in enumerate(stde2e_io_time_bars):
    ax.text(i - width*0.75, v*0.5, f'{v:.0f}', ha='center', va='bottom')

for i, v in enumerate(std_io_time_bars):
    ax.text(i + width*0.25, v*0.5, f'{v:.0f}', ha='center', va='center', color='black')

for i, v in enumerate(middleware_overhead_bars):
    middle_y = std_io_time_bars[i] + v*0.5
    ax.text(i + width*0.25, middle_y, f'{v:.0f}', ha='center', va='center', color='black')

for i, v in enumerate(p2m_overhead_bars):
    middle_y = std_io_time_bars[i] + middleware_overhead_bars[i] + v*0.5
    ax.text(i + width*0.25, middle_y, f'{v:.0f}', ha='center', va='center', color='black')



# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()